##GAN
This is a dry run and first implementation of GAN!

Just the beginning!

In [ ]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Image Dimensions(MNIST Dataset)

1x28x28

In [ ]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

#GENERATOR FUNCTION

In [ ]:
def build_generator():
  noise_shape = (100,) #1D array of size 100 (latent vector / noise)
  model = Sequential()

  model.add(Dense(256, input_shape=noise_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(2048))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  print(model.summary())

  noise = Input(shape=noise_shape)
  img = model(noise)    #Generated image

  return Model(noise, img)

Alpha — α is a hyperparameter which controls the underlying value to which the
function saturates negatives network inputs.
Momentum — Speed up the training

#DISCRIMINATOR FUNCTION

In [ ]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

The validity is the Discriminator’s guess of input being real or not.


In [ ]:
def train(epochs, batch_size=128, save_interval=50):
  # Load the dataset
  (X_train, _), (_, _) = mnist.load_data()
  # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
  X_train = (X_train.astype(np.float32) - 127.5) / 127.5

  #Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
  #This would have 
  X_train = np.expand_dims(X_train, axis=3)
  half_batch = int(batch_size / 2)
  
  for epoch in range(epochs):
        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective. 
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))


        #take average loss from real and fake images. 
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, 100)) 

        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        # job of fooling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)

        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        
        if epoch % save_interval == 0:
            save_imgs(epoch)


In [ ]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("/content/gdrive/My Drive/GAN/mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view


##############################################################################

#Let us also define our optimizer for easy use later on.
#That way if you change your mind, you can change it easily here
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

               
#Binary cross entropy loss function
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

#build and compile our Discriminator, pick the loss function


generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)


z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)


discriminator.trainable = False  


valid = discriminator(img)  #Validity check on the generated image




combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=3000, batch_size=32, save_interval=50)


#Compare with GAN4

generator.save('generator_model.h5')  #Test the model on GAN4_predict...


Streaming output truncated to the last 5000 lines.
524 [D loss: 0.726717, acc.: 56.25%] [G loss: 1.495501]
1/1 [==============================] - 0s 33ms/step
525 [D loss: 0.519850, acc.: 78.12%] [G loss: 2.493309]
1/1 [==============================] - 0s 29ms/step
526 [D loss: 0.641646, acc.: 56.25%] [G loss: 1.634462]
1/1 [==============================] - 0s 34ms/step
527 [D loss: 0.678058, acc.: 40.62%] [G loss: 1.976399]
1/1 [==============================] - 0s 43ms/step
528 [D loss: 0.606360, acc.: 75.00%] [G loss: 1.691685]
1/1 [==============================] - 0s 35ms/step
529 [D loss: 0.648536, acc.: 71.88%] [G loss: 1.268062]
1/1 [==============================] - 0s 34ms/step
530 [D loss: 0.624103, acc.: 75.00%] [G loss: 1.393806]
1/1 [==============================] - 0s 33ms/step
531 [D loss: 0.715029, acc.: 50.00%] [G loss: 1.747216]
1/1 [==============================] - 0s 46ms/step
532 [D loss: 0.532983, acc.: 68.75%] [G loss: 1.751121]
1/1 [========================